In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

As explained in the report, hardcode latencies for critical points and establish a function to obtain the CPU latency between Core #0 and Core#p. Such function trivially interpolates the CPU latency assigning the same value found at the critical point until the next one is reached. 

In [2]:
latencies = {
  "1": 0.14,   # CCX
  "4": 0.35,   # CCD
  "8": 0.35,   # NUMA
  "32": 0.41,  # socket 
  "64": 0.65,  # node 
  "128": 1.82, # cluster
}
def get_latency(p):
  for k in list(latencies)[::-1]:
    if int(k) <= p:
      return latencies[k]
  return latencies[list(latencies.keys())[0]]
df = pd.DataFrame(data={
  "cpu": range(1, 256 + 1),
  "avg_latency": [get_latency(i) for i in range(256)]
}).dropna()

Derive the time spent for a double ring of np processes by converting the 0 -> p latencies into p_i -> p_(i + 1) latencies. As explained in the report, this is needed due to the implementation of the double ring algorithm, based on the exchanges between neighbouring processes.

In [3]:
def get_double_ring_time(np):
  def get_latency(a, b):
    if a // 4 == b // 4:
      return latencies["1"]
    if a // 8 == b // 8:
      return latencies["4"]
    if a // 16 == b // 16:
      return latencies["8"]
    if a // 64 == b // 64:
      return latencies["32"]
    if a // 128 == b // 128:
      return latencies["64"]
    return latencies["128"]
  return sum([get_latency(i, (i + 1) % np) for i in range(np - 1)])

Load measured values for the Barrier collective using the Double Ring algorithm and calculate its estimate based on an empirical formula, as explained in the report.

In [4]:
actual = pd.read_csv("data/20240222_205355_epyc_barrier2_msize2.txt")
df["actual_barrier"] = actual
df["estimate_barrier"] = df.apply(lambda e: get_double_ring_time(int(e["cpu"] - 1)), axis=1)

In [5]:
df

,cpu,avg_latency,actual_barrier,estimate_barrier
0,1,0.14,0.18,0.00
1,2,0.14,0.52,0.00
2,3,0.14,0.45,0.14
3,4,0.14,0.82,0.28
4,5,0.35,1.03,0.42
...,...,...,...,...
251,252,1.82,48.06,50.81
252,253,1.82,43.76,50.95
253,254,1.82,52.34,51.30
254,255,1.82,14.36,51.44


In [6]:
fig = px.scatter(df, x="cpu", y="actual_barrier")
fig.add_trace(go.Scatter(x=df["cpu"], y=df["estimate_barrier"], mode="lines", name="est_barr"))